In [158]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow.keras as keras 
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout

In [95]:
folder = os.getcwd().replace('\\','/')+'/nlp-getting-started/'

In [96]:
os.listdir(folder)

['sample_submission.csv', 'test.csv', 'train.csv']

In [97]:
pd.set_option('display.max_colwidth', None)

In [98]:
df_train = pd.read_csv(folder+'train.csv', sep=',')
df_test = pd.read_csv(folder+'test.csv', sep=',')

In [99]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7613 entries, 0 to 7612
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        7613 non-null   int64 
 1   keyword   7552 non-null   object
 2   location  5080 non-null   object
 3   text      7613 non-null   object
 4   target    7613 non-null   int64 
dtypes: int64(2), object(3)
memory usage: 297.5+ KB


In [100]:
df_train.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake May ALLAH Forgive us all,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are being notified by officers. No other evacuation or shelter in place orders are expected,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation orders in California",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as smoke from #wildfires pours into a school,1


In [101]:
df_train = df_train.drop(['keyword','location'], axis=1)
df_test = df_test.drop(['keyword','location'], axis=1)

In [102]:
df_train.head(10)

,id,text,target
0,1,Our Deeds are the Reason of this #earthquake May ALLAH Forgive us all,1
1,4,Forest fire near La Ronge Sask. Canada,1
2,5,All residents asked to 'shelter in place' are being notified by officers. No other evacuation or shelter in place orders are expected,1
3,6,"13,000 people receive #wildfires evacuation orders in California",1
4,7,Just got sent this photo from Ruby #Alaska as smoke from #wildfires pours into a school,1
5,8,#RockyFire Update => California Hwy. 20 closed in both directions due to Lake County fire - #CAfire #wildfires,1
6,10,"#flood #disaster Heavy rain causes flash flooding of streets in Manitou, Colorado Springs areas",1
7,13,I'm on top of the hill and I can see a fire in the woods...,1
8,14,There's an emergency evacuation happening now in the building across the street,1
9,15,I'm afraid that the tornado is coming to our area...,1


In [103]:
x = df_train.sort_values(by= ['id'], ascending=True)
x_test = df_test.sort_values(by= ['id'], ascending=True)
#x.describe()

In [104]:
x.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7613 entries, 0 to 7612
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      7613 non-null   int64 
 1   text    7613 non-null   object
 2   target  7613 non-null   int64 
dtypes: int64(2), object(1)
memory usage: 237.9+ KB


In [105]:
#Retrieve the text from the dataframe as a numpy array
twitts = x.loc[:,x.columns=='text'].values.flatten()
twitts_test = x_test.loc[:,x_test.columns=='text'].values.flatten()

In [122]:
print(twitts[:10])

['Our Deeds are the Reason of this #earthquake May ALLAH Forgive us all'
 'Forest fire near La Ronge Sask. Canada'
 "All residents asked to 'shelter in place' are being notified by officers. No other evacuation or shelter in place orders are expected"
 '13,000 people receive #wildfires evacuation orders in California '
 'Just got sent this photo from Ruby #Alaska as smoke from #wildfires pours into a school '
 '#RockyFire Update => California Hwy. 20 closed in both directions due to Lake County fire - #CAfire #wildfires'
 '#flood #disaster Heavy rain causes flash flooding of streets in Manitou, Colorado Springs areas'
 "I'm on top of the hill and I can see a fire in the woods..."
 "There's an emergency evacuation happening now in the building across the street"
 "I'm afraid that the tornado is coming to our area..."]


In [123]:
twitts.shape

(7613,)

In [125]:
t = Tokenizer()
def preprocess(text):
    seqs = t.fit_on_texts(text)
    return seqs

In [126]:
#fit tokenizer on training set:
tokens=preprocess(twitts)

In [127]:
#text to sequences
X_train = t.texts_to_sequences(twitts)
X_test = t.texts_to_sequences(twitts_test)

In [132]:
#pad sequences
X_train = np.array(X_train)
X_test = np.array(X_test)
X_train = keras.preprocessing.sequence.pad_sequences(X_train)
X_test = keras.preprocessing.sequence.pad_sequences(X_test)

In [134]:
print(X_train[0])

[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0  119 4633   24    4  868    8   21  263
  138 1619 4634   89   40]


In [136]:
#size of the vocabulary
vocab_size = len(t.word_index)
print('Size of vocabulary:', vocab_size)

Size of vocabulary: 22700


In [148]:
# target training variable:
y_train = df_train.target.values

In [151]:
print(y_train[:50])

[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 1 0 0 0
 1 1 0 0 0 0 0 0 0 1 0 0 0]


In [3]:
#create the model
model = Sequential()
model.add(Embedding(input_dim = vocab_size+2, output_dim = 32, name='Embedding'))
model.add(LSTM(64)) 
model.add(Dropout(0.5))
model.add(Dense(64, activation = 'relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='adam', loss="binary_crossentropy", metrics=["accuracy"])
model.summary()

NameError: name 'Sequential' is not defined

In [2]:
history = model.fit(X_train, y_train, shuffle=True, epochs=5, batch_size=64)

NameError: name 'model' is not defined

In [163]:
score = model.evaluate(X_train, y_train)

7613/7613 [==============================] - 7s 923us/sample - loss: 0.0179 - accuracy: 0.9947


In [ ]:
# function to predict whether a twitt refers to a real disaster or not (1 real, 0 otherwise):
def predict_message(pred_text):
    sequence = tokenizer.texts_to_sequences([pred_text])
    sequences = keras.preprocessing.sequence.pad_sequences(sequence)
    predictions = model.predict(sequences)[0]
    predicted = [np.max(predictions), int_label[np.argmax(predictions)]]

    return (predicted)

pred_text = "how are you doing today?"

prediction = predict_message(pred_text)
print(prediction)

In [202]:
preds = np.where(model.predict(X_test)>0.5,1.0,0.0).flatten().astype('int32')

In [206]:
print(preds[20:30])

[0 0 1 1 0 1 0 0 0 1]


In [207]:
print(twitts_test[20:30])

['beware world ablaze sierra leone &amp; guap.'
 'Burning Man Ablaze! by Turban Diva http://t.co/hodWosAmWS via @Etsy'
 "Not a diss song. People will take 1 thing and run with it. Smh it's an eye opener though. He is about 2 set the game ablaze @CyhiThePrynce"
 'Rape victim dies as she sets herself ablaze: A 16-year-old girl died of burn injuries as she set herself ablaze\x89Û_ http://t.co/UK8hNrbOob'
 'SETTING MYSELF ABLAZE http://t.co/6vMe7P5XhC'
 '@CTVToronto the bins in front of the field by my house wer set ablaze the other day flames went rite up the hydro pole wonder if it was him'
 '#nowplaying Alfons - Ablaze 2015 on Puls Radio #pulsradio http://t.co/aA5BJgWfDv'
 "'Burning Rahm': Let's hope City Hall builds a giant wooden mayoral effigy 100 feet tall &amp; sets it ablaze. http://t.co/kFo2mksn6Y @John_Kass"
 '@PhilippaEilhart @DhuBlath hurt but her eyes ablaze with insulted anger.'
 'Accident cleared in #PaTurnpike on PATP EB between PA-18 and Cranberry slow back to #traffic ht

In [208]:
fits = np.where(model.predict(X_train)>0.5,1.0,0.0).flatten().astype('int32')

In [209]:
print(fits[10:20])

[1 1 1 1 1 0 0 0 0 0]


In [201]:
print(twitts[10:20])

['Three people died from the heat wave so far'
 'Haha South Tampa is getting flooded hah- WAIT A SECOND I LIVE IN SOUTH TAMPA WHAT AM I GONNA DO WHAT AM I GONNA DO FVCK #flooding'
 "#raining #flooding #Florida #TampaBay #Tampa 18 or 19 days. I've lost count "
 '#Flood in Bago Myanmar #We arrived Bago'
 'Damage to school bus on 80 in multi car crash #BREAKING '
 "What's up man?" 'I love fruits' 'Summer is lovely' 'My car is so fast'
 'What a goooooooaaaaaal!!!!!!']


In [260]:
results = {'text':twitts_test, 'target':preds}
df_results = pd.DataFrame(results)

In [285]:
df_results.head()

,text,target
0,Just happened a terrible car crash,1
1,"Heard about #earthquake is different cities, stay safe everyone.",1
2,"there is a forest fire at spot pond, geese are fleeing across the street, I cannot save them all",1
3,Apocalypse lighting. #Spokane #wildfires,1
4,Typhoon Soudelor kills 28 in China and Taiwan,1


In [286]:
df_test.head()

,id,text
0,0,Just happened a terrible car crash
1,2,"Heard about #earthquake is different cities, stay safe everyone."
2,3,"there is a forest fire at spot pond, geese are fleeing across the street, I cannot save them all"
3,9,Apocalypse lighting. #Spokane #wildfires
4,11,Typhoon Soudelor kills 28 in China and Taiwan


In [316]:
submission = pd.merge(df_test, df_results, on='text').drop_duplicates()

In [317]:
submission

,id,text,target
0,0,Just happened a terrible car crash,1
1,2,"Heard about #earthquake is different cities, stay safe everyone.",1
2,3,"there is a forest fire at spot pond, geese are fleeing across the street, I cannot save them all",1
3,9,Apocalypse lighting. #Spokane #wildfires,1
4,11,Typhoon Soudelor kills 28 in China and Taiwan,1
...,...,...,...
3300,10861,EARTHQUAKE SAFETY LOS ANGELES ÛÒ SAFETY FASTENERS XrWn,1
3301,10865,Storm in RI worse than last hurricane. My city&amp;3others hardest hit. My yard looks like it was bombed. Around 20000K still without power,1
3302,10868,Green Line derailment in Chicago http://t.co/UtbXLcBIuY,1
3303,10874,MEG issues Hazardous Weather Outlook (HWO) http://t.co/3X6RBQJHn3,1


In [318]:
submission.drop(['text'],axis=1).to_csv('submission.csv', index=False)

In [319]:
submission.describe()

,id,target
count,3263.000000,3263.000000
mean,5427.152927,0.403923
std,3146.427221,0.490758
min,0.000000,0.000000
25%,2683.000000,0.000000
50%,5500.000000,0.000000
75%,8176.000000,1.000000
max,10875.000000,1.000000


In [321]:
submission.set_index('id').head(20)

,text,target
id,,
0,Just happened a terrible car crash,1
2,"Heard about #earthquake is different cities, stay safe everyone.",1
3,"there is a forest fire at spot pond, geese are fleeing across the street, I cannot save them all",1
9,Apocalypse lighting. #Spokane #wildfires,1
11,Typhoon Soudelor kills 28 in China and Taiwan,1
12,We're shaking...It's an earthquake,1
21,"They'd probably still show more life than Arsenal did yesterday, eh? EH?",0
22,Hey! How are you?,0
27,What a nice hat?,0
